<a href="https://colab.research.google.com/github/hthompso15/Network_stats_data_project/blob/main/NetworkStatsFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [5]:
! git clone https://github.com/jg-you/stat-6990-dataset-structure.git

Cloning into 'stat-6990-dataset-structure'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 25 (delta 2), reused 22 (delta 2), pack-reused 0
Receiving objects: 100% (25/25), 117.66 KiB | 661.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

In [ ]:
# Colab uses a Python install that deviates from the system's! Bad collab! We need some workarounds.
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

In [1]:
import graph_tool.all as gt
import networkx as nx
import random
import itertools as it
import numpy as np


g = gt.collection.ns["us_air_traffic"]
print('done')

done


In [15]:
# Get node properties
node_properties = {key: g.vertex_properties[key] for key in g.vertex_properties.keys()}

# Get edge properties
edge_properties = {key: g.edge_properties[key] for key in g.edge_properties.keys()}

print("Node properties:")
for key, prop in node_properties.items():
    print(key, prop)

print("\nEdge properties:")
for key, prop in edge_properties.items():
    print(key, prop)


Node properties:
airport_code <VertexPropertyMap object with value type 'string', for Graph 0x78f7bcb3b8b0, at 0x78f7bcb383a0>
airport_id <VertexPropertyMap object with value type 'int16_t', for Graph 0x78f7bcb3b8b0, at 0x78f7bcb3b880>
airport_seq_id <VertexPropertyMap object with value type 'int32_t', for Graph 0x78f7bcb3b8b0, at 0x78f7bcb3a830>
city_market_id <VertexPropertyMap object with value type 'int32_t', for Graph 0x78f7bcb3b8b0, at 0x78f7bcb38fa0>
origin <VertexPropertyMap object with value type 'string', for Graph 0x78f7bcb3b8b0, at 0x78f7bcb3a620>
city_name <VertexPropertyMap object with value type 'string', for Graph 0x78f7bcb3b8b0, at 0x78f7bcb39c30>
state_abr <VertexPropertyMap object with value type 'string', for Graph 0x78f7bcb3b8b0, at 0x78f7bcb3a6e0>
state_fips <VertexPropertyMap object with value type 'string', for Graph 0x78f7bcb3b8b0, at 0x78f7bcb3bfd0>
state_nm <VertexPropertyMap object with value type 'string', for Graph 0x78f7bcb3b8b0, at 0x78f7bcb3aa40>
wac <V

In [18]:
g.save("df_data.gml")

In [ ]:
pos = gt.sfdp_layout(g)

# Visualize the graph
gt.graph_draw(g, pos=pos, output="graph_visualization.png")